In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 2 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Stabilité des méthodes
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

# Stabilité des méthodes

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

def compute_divided_diff_coef(x, y):
    n = x.size
    coef = y.copy()
    for i in range(1,n):
        coef[i:] = (coef[i:] - coef[i-1:-1])/(x[i:] - x[:-i])
    return coef

def newton_interp(xi, yi, x):
    coef = compute_divided_diff_coef(xi, yi)
    n = xi.size
    p = np.zeros(x.size)
    for i in range(n-1,0,-1):
        p = (coef[i]+p) * (x-xi[i-1])
    p = p +  coef[0]
    return p

def improved_lag_interp(xk, yk, x):
    n = xk.size
    l = np.ones(x.size)
    p = np.zeros(x.size)
    
    wk = np.zeros(n) 
    for k in range(n):
        wk[k] = 1. / (np.prod( xk[k] - xk[np.arange(n)!=k] ))
    
    mask = np.in1d(x, xk[:])
    p[np.where(mask)] = yk[np.where(np.in1d(xk, x))]
    mask = np.invert(mask)    

    for k in range(n):
        l[mask] *= x[mask] - xk[k]
        p[mask] += (wk[k]/(x[mask] - xk[k])) * yk[k]
        
    return l*p

def cheb_points(xmin, xmax, n):
    x = np.zeros(n+1)
    for i in range(n+1):
        x[i] = (xmin+xmax)/2 + ((xmax-xmin)/2) * np.cos(((2*i+1)*np.pi)/(2*n + 2))
    return x

def noise(eps, x): 
    return eps * (2*np.random.rand(x.size)-1) 

def lagrange(k, xk, x):
    lag = np.ones(x.size)
    for j in range(xk.size):
        if (j!=k): lag *= (x-xk[j])/(xk[k]-xk[j])
    return lag

def compute_cond(xk, yk, pn, x):
    cond = np.zeros(x.size)
    for k in range(xk.size):
        cond += np.abs(lagrange(k, xk, x) * yk[k]) / np.abs(pn)
    return cond

On considère l’approximation polynomiale de la fonction $f(x) = \sin(\pi x)$ sur $[-1,1]$.

In [ ]:
def f(x):
    return np.sin(np.pi*x)

xmin = -1.; xmax = 1.

## Méthode de Newton

On interpole la fonction $f(x)$ en utilisant des points de Tchebychev.

In [ ]:
# x use to display f and pn
xplot = np.linspace(xmin, xmax, 500)

# maximal degree of Newton interpolating polynomial
n_max = 71

# array of degree
n = np.arange(19, n_max+1, 2)

# compute for each degree xi, yi and pn
xk = []; yk = []; pn = []
for i, ni in enumerate(n):
    xk.append(cheb_points(xmin, xmax, ni))
    yk.append(f(xk[i]))
    pn.append(newton_interp(xk[i], yk[i], xplot))
    
# Create figure
fig = go.Figure(layout_title="Approximation polynomiale avec la méthode de Newton")

# add f(x) plot
fig.add_trace(go.Scatter(visible=True, x=xplot, y=f(xplot), name="f(x)"))

# add yi and pn(x) invisible plots
for i, ni in enumerate(n):
    fig.add_trace(go.Scatter(visible=False, x=xplot, y=pn[i], name=f"p{ni}(x)"))
    fig.add_trace(go.Scatter(visible=False, x=xk[i], y=yk[i], mode='markers', name="pts d'interpolation"))

# Make plot visible for n[0]
fig.data[1].visible = True
fig.data[2].visible = True

# Create and add slider
steps = []
for i, ni in enumerate(n):
    step = dict(method="update", label = f" {ni+1}",
                args=[{"visible": [el==0 or el==2*i+1 or el==2*i+2 for el in range(len(fig.data))]}])
    steps.append(step)
        
sliders = [dict(currentvalue={"prefix": "nb points: "}, steps=steps)]

fig.update_layout(sliders=sliders)
fig.update_yaxes(range=[-1.1, 1.1])

fig.show()

Pour cette répartition de points, le conditionnement du problème est bon. Les erreurs constatées à partir de 68 points sont dûes à la stabilité de la méthode utilisées pour interpoler.

## Première forme de la formule d’interpolation barycentrique

On interpole la fonction $f(x)$ en utilisant des points de Tchebychev.  

In [ ]:
# maximal degree of Newton interpolating polynomial
n_max = 520

# array of degree
n = np.arange(19, n_max+1, 40)

# x use to display f and pn
xplot = np.linspace(xmin, xmax, 2000)

# compute for each degree xi, yi and pn
xk = []; yk = []; pn = []
for i, ni in enumerate(n):
    xk.append(cheb_points(xmin, xmax, ni))
    yk.append(f(xk[i]))
    pn.append(improved_lag_interp(xk[i], yk[i], xplot))
    
# Create figure
fig = go.Figure(layout_title="Approximation polynomiale utilisant des points Tchebychev")

# add f(x) plot
fig.add_trace(go.Scatter(visible=True, x=xplot, y=f(xplot), name="f(x)"))

# add yi and pn(x) invisible plots
for i, ni in enumerate(n):
    fig.add_trace(go.Scatter(visible=False, x=xplot, y=pn[i], name=f"p{ni}(x)"))
    fig.add_trace(go.Scatter(visible=False, x=xk[i], y=yk[i], mode='markers', name="pts d'interpolation"))

# Make plot visible for n[0]
fig.data[1].visible = True
fig.data[2].visible = True

# Create and add slider
steps = []
for i, ni in enumerate(n):
    step = dict(method="update", label = f" {ni+1}",
                args=[{"visible": [el==0 or el==2*i+1 or el==2*i+2 for el in range(len(fig.data))]}])
    steps.append(step)
        
sliders = [dict(currentvalue={"prefix": "nb points: "}, steps=steps)]

fig.update_layout(sliders=sliders)
fig.update_yaxes(range=[-1.1, 1.1])

fig.show()

Dans ce cas, on peut augmenter les nombre de points sans observer d'erreur. C'est le comportement attendu de l'utilisation d'un algorithme stable pour un bon conditionnement local du problème.